# Aнализ зарплат в IT

In [41]:
import pandas as pd 
import numpy as np
import matplotlib as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from collections import Counter

## EDA

In [2]:
df = pd.read_csv('DataScienceJobs.csv')
df.head()

name          address  \
0  Начинающий специалист по работе с нейросетям  Санкт-Петербург   
1                          Junior Data Engineer  Нижний Новгород   
2                                    ML-инженер           Москва   
3                                Data Scientist           Москва   
4                     Python Developer (Junior)            Томск   

           experience                                             skills  \
0  От 1 года до 3 лет  Разработка технических заданий, Python, Сбор и...   
1  От 1 года до 3 лет                                   SQL, jvm, Hadoop   
2           Нет опыта  Знания математики, алгоритмов, структур данных...   
3  От 1 года до 3 лет  Python, Machine Learning, Математическое модел...   
4  От 1 года до 3 лет                            Python, Git, ООП, MySQL   

   sal_from  sal_to  
0     55000   60000  
1    150000  200000  
2    150000  170000  
3    180000  200000  
4     60000  110000

In [3]:
df['name'].value_counts() ## построим частотную таблицу

name
Data Scientist                                       36
Data Engineer                                        18
Senior Data Scientist                                 8
ML Engineer                                           6
Machine Learning Engineer                             6
                                                     ..
Senior Data Scientist CV (Generative design)          1
Ведущий специалист по разработке скоринговых карт     1
Lead data scientist (Анализ цены и Trade in)          1
Data engineer (middle+)                               1
Автор дисциплины "Задачи NLP в машинном обучении"     1
Name: count, Length: 246, dtype: int64

In [4]:
df['name'].nunique() # посчитаем количество уникальных названий вакансий в наборе

246

In [5]:
df['name'].mode() # самое часто встречающееся название вакансии, мода

0    Data Scientist
Name: name, dtype: object

In [6]:
# найдем вакансии, в названии которых встречается Data Scientist или Data scientist или data scientist

df[(df['name'].str.contains('Data Scientist')) 
   | (df['name'].str.contains('Data scientist')) 
   | (df['name'].str.contains('data scientist'))].shape

(151, 6)

In [7]:
df['address'].value_counts(normalize=True) * 100 # частотная таблица вакансий в городах в процентах

address
Москва                52.222222
Санкт-Петербург       11.111111
Московская область     7.777778
Новосибирск            7.500000
Казань                 4.166667
Нижний Новгород        4.166667
Екатеринбург           4.166667
Тюмень                 3.055556
Томск                  2.777778
Сочи                   2.222222
Владивосток            0.555556
Астрахань              0.277778
Name: proportion, dtype: float64

In [9]:
df['experience'].value_counts() #Посмотрим на требуемый опыт

experience
От 3 до 6 лет         199
От 1 года до 3 лет    140
Более 6 лет            12
Нет опыта               9
Name: count, dtype: int64

In [17]:
df.describe() #описательные статистики

sal_from         sal_to
count     360.000000     360.000000
mean   182694.444444  231416.666667
std     70581.279109   79938.651617
min     45000.000000   55000.000000
25%    120000.000000  170000.000000
50%    180000.000000  225000.000000
75%    242500.000000  290000.000000
max    350000.000000  570000.000000

## Визуализация

### Вопрос 1. В каких городах больше всего вакансий для Data Scientists?

In [24]:
top_cities = df['address'].value_counts()[:5]
top_cities = top_cities.to_frame().reset_index()
top_cities

address  count
0              Москва    188
1     Санкт-Петербург     40
2  Московская область     28
3         Новосибирск     27
4              Казань     15

In [26]:
fig = px.pie(top_cities, values= 'count', names = 'address', title='Самые популярные города')
fig

### Вопрос 2. Какие навыки требуются на рынке? Чему учиться?

In [39]:
skills = df['skills'].str.cat(sep = ' ').lower().replace(',', '').replace(';', '').split()
skills = [skill for skill in skills if len(skill) > 1]

skills_vis = Counter(skills).most_common(15)
sk = pd.DataFrame(skills_vis, columns = ['skill', 'num'])
sk['skill'] = sk['skill'].str.upper() # приведем все названия к верхнему регистру
sk


skill  num
0     PYTHON  255
1        SQL  165
2     ДАННЫХ   90
3       DATA   87
4         ML   82
5   LEARNING   80
6       ОПЫТ   76
7    MACHINE   64
8    PYTORCH   63
9     РАБОТЫ   54
10    PANDAS   51
11    АНАЛИЗ   47
12       NLP   47
13       GIT   42
14     SPARK   42

In [42]:
trace0 = go.Pie(
    name = "",
    values = sk['num'],
    labels = sk['skill'],
    hovertemplate = "<b>%{label}</b>\n <br>Доля: %{percent} </br>Количество: %{value} упоминаний"
)
layout= go.Layout(
    title= 'Топ-15 ключевых слов в описаниях вакансий'
)
fig = go.Figure(data = [trace0], layout = layout)
fig

### Вопрос 3. Какие зарплаты на рынке? С каким опытом берут на работу?

In [27]:
experience = df['experience'].value_counts()
experience = experience.to_frame().reset_index()
experience

experience  count
0       От 3 до 6 лет    199
1  От 1 года до 3 лет    140
2         Более 6 лет     12
3           Нет опыта      9

In [43]:
df.describe().map("{0:,.2f}".format)

sal_from      sal_to
count      360.00      360.00
mean   182,694.44  231,416.67
std     70,581.28   79,938.65
min     45,000.00   55,000.00
25%    120,000.00  170,000.00
50%    180,000.00  225,000.00
75%    242,500.00  290,000.00
max    350,000.00  570,000.00

In [44]:
# разобьем длинные строки навыков на строки покороче, чтобы они лучше отрисовывались

import textwrap
df.skills = df.skills.apply(lambda txt: '<br>'.join(textwrap.wrap(txt, width=50)))

In [46]:
fig = px.scatter(data_frame=df, 
                 x='sal_from', 
                 y='sal_to', 
                 color='experience',
                 title='Распределение заработных плат в зависимости от опыта',
                 hover_name='name', 
                 hover_data=['sal_to', 'sal_from', 'skills'], 
                 color_discrete_sequence=px.colors.qualitative.Plotly,
                 category_orders={"experience":
                                  ["Нет опыта", "От 1 года до 3 лет", "От 3 до 6 лет", "Более 6 лет"]},
               labels={'experience': 'Опыт', 
                       'sal_to': 'Верхняя вилка зарплаты', 
                       'sal_from': 'Нижняя вилка зарплаты',
                       'skills': 'Требуемые навыки'}
              ).update_traces(marker_size=10)
fig